# Cosmos DB in Fabric #
# Credit Card Fraud Detection Sample – Part 3: Generate Pending Transactions #

This section generates pending transactions that will be consumed by spark stream read in Part 2 via changefeed. Here we simply add a pending transaction item with expected values into the PendingCCTransaction container.

### Prerequisites ###
Before running this notebook, ensure you have:

- A **Cosmos DB artifact** created in Microsoft Fabric.

- One container:

    - **PendingCCTransactions** – Stores pending credit card transaction records to be processed later.

In [ ]:
%pip install azure-core azure-cosmos

In [ ]:
import base64, json
import uuid
import random
import time
import math
import numpy as np
from datetime import datetime, timezone

from typing import Any, Optional, List, Dict, Tuple

#from azure.cosmos.aio import CosmosClient why aio
from azure.cosmos import CosmosClient, PartitionKey, ThroughputProperties
from azure.core.credentials import TokenCredential, AccessToken


COSMOS_ENDPOINT = '<COSMOS_ENDPOINT>' # The Cosmos DB artifact endpoint from the artifact settings
COSMOS_DATABASE_NAME = '<COSMOS_DATABASE_NAME>' # The Cosmos DB artifact name is the database name
COSMOS_PENDING_TRANSACTION_CONTAINER_NAME = 'PendingCCTransactions'
COSMOS_CC_CONTAINER_NAME = 'CreditCards'


In [ ]:
## Authentication Class

class FabricTokenCredential(TokenCredential):
    """Token credential for Fabric Cosmos DB access with automatic refresh and retry logic."""
    
    def get_token(self, *scopes: str, claims: Optional[str] = None, tenant_id: Optional[str] = None,
                  enable_cae: bool = False, **kwargs: Any) -> AccessToken:
        access_token = notebookutils.credentials.getToken("https://cosmos.azure.com/.default")
        parts = access_token.split(".")
        if len(parts) < 2:
            raise ValueError("Invalid JWT format")
        payload_b64 = parts[1]
        # Fix padding
        padding = (-len(payload_b64)) % 4
        if padding:
            payload_b64 += "=" * padding
        payload_json = base64.urlsafe_b64decode(payload_b64.encode("utf-8")).decode("utf-8")
        payload = json.loads(payload_json)
        exp = payload.get("exp")
        if exp is None:
            raise ValueError("exp claim missing in token")
        return AccessToken(token=access_token, expires_on=exp)

In [ ]:
# Initialize Cosmos DB cosmos client
COSMOS_CLIENT = CosmosClient(COSMOS_ENDPOINT, FabricTokenCredential())

# Initialize Cosmos DB database client
DATABASE_CLIENT = COSMOS_CLIENT.get_database_client(COSMOS_DATABASE_NAME)

# Initialize the Pending_txns_container
pending_txns_container = DATABASE_CLIENT.get_container_client(COSMOS_PENDING_TRANSACTION_CONTAINER_NAME)

In [ ]:

# ID ranges
CUSTOMER_IDS = [f"U{n:04d}" for n in range(1, 101)]  # U0001..U0100
CARD_IDS     = [f"C{n:04d}" for n in range(1, 101)] # C0001..C0100"


In [ ]:

# Merchants & Locations
merchants = [
    # Retail & E‑Com
    "Amazon","Walmart","Target","Best Buy","Costco","Home Depot","Lowe's",
    "Macy's","Nordstrom","Kohl's","eBay","Wayfair","Etsy","AliExpress","Shein",
    "Sam's Club","BJ's Wholesale","Bed Bath & Beyond",

    # Food & Beverage
    "Starbucks","Dunkin'","McDonald's","Subway","Chipotle","Panera Bread",
    "Domino's Pizza","Pizza Hut","Chick-fil-A","Burger King","Taco Bell",
    "KFC","Popeyes","Shake Shack","Five Guys",

    # Tech & Electronics
    "Apple Store","Microsoft","Google","Samsung","Dell","HP","Lenovo",
    "Sony","Asus","Acer","Nvidia",

    # Fashion & Sports
    "Nike","Adidas","Under Armour","Lululemon","Zara","H&M","Gap",
    "Old Navy","Uniqlo","Forever 21","Victoria's Secret",

    # Entertainment & Streaming
    "Netflix","Hulu","Disney+","Spotify","YouTube Premium","Amazon Prime Video",
    "HBO Max","Peacock","Paramount+","Apple TV+","Crunchyroll",

    # Travel & Transport
    "Uber","Lyft","Airbnb","Expedia","Booking.com","Delta Airlines",
    "United Airlines","American Airlines","Southwest Airlines","Marriott",
    "Hilton","Hyatt",

    # Financial & Services
    "PayPal","Venmo","Stripe","Square","Cash App","Zelle",

    # Grocery & Pharmacy
    "Kroger","Safeway","Albertsons","Publix","Trader Joe's","Whole Foods",
    "CVS","Walgreens","Rite Aid",

    # Home & Lifestyle
    "IKEA","Ashley Furniture","Crate & Barrel","Williams Sonoma",

    # Luxury & Jewelry
    "Tiffany & Co.","Cartier","Rolex","Gucci","Louis Vuitton","Prada",
]


US_STATES = [
    "Alabama","Alaska","Arizona","Arkansas","California","Colorado","Connecticut","Delaware","Florida","Georgia",
    "Hawaii","Idaho","Illinois","Indiana","Iowa","Kansas","Kentucky","Louisiana","Maine","Maryland","Massachusetts",
    "Michigan","Minnesota","Mississippi","Missouri","Montana","Nebraska","Nevada","New Hampshire","New Jersey",
    "New Mexico","New York","North Carolina","North Dakota","Ohio","Oklahoma","Oregon","Pennsylvania","Rhode Island",
    "South Carolina","South Dakota","Tennessee","Texas","Utah","Vermont","Virginia","Washington","West Virginia",
    "Wisconsin","Wyoming"
]

locations = US_STATES + [
    # Countries
    "Canada","Mexico","Brazil","Argentina","United Kingdom","France","Germany","Italy","Spain",
    "Netherlands","Sweden","Norway","Denmark","Finland","Poland","Switzerland","Austria","Belgium",
    "Australia","New Zealand","Japan","South Korea","China","India","Singapore","Thailand","Vietnam",
    "South Africa","Nigeria","Egypt","Saudi Arabia","Philippines","South Africa","Nigeria","Egypt",
    "Saudi Arabia","United Arab Emirates","Turkey","Israel","Russia"]


In [ ]:

# CELL — Generate and insert random pending transactions

# Assume these lists are defined above:
# CUSTOMER_IDS = [f"U{n:04d}" for n in range(1, 101)]
# CARD_IDS     = [f"C{n:04d}" for n in range(1, 101)]
# merchants    = [...]  # your merchant list
# locations    = US_STATES + [list of countries]

def add_random_pending_transactions(n: int = 50):
    for _ in range(n):
        # Pick a matching card/customer pair by index
        idx = random.randint(0, len(CARD_IDS) - 1)
        card_id = CARD_IDS[idx]
        customer_id = CUSTOMER_IDS[idx]

        # Random merchant, location, and amount
        merchant = random.choice(merchants)
        location = random.choice(locations)
        amount = round(random.uniform(5.0, 2000.0), 2)

        # Build the document
        doc = {
            "id": str(uuid.uuid4()),
            "type": "transaction",
            "card_id": card_id,
            "customer_id": customer_id,
            "merchant": merchant,
            "location": location,
            "amount": amount,
            "timestamp": datetime.now().replace(microsecond=0).isoformat()
        }

        # Insert into PendingCCTransactions
        pending_txns_container.create_item(doc)

    print(f"Inserted {n} pending transactions.")


In [ ]:
while True:
    add_random_pending_transactions(5)
    time.sleep(60)

In [ ]:
# Manually add a new pending transaction
doc = {
            "id": str(uuid.uuid4()),
            "type": "transaction",
            "card_id": card_id,
            "customer_id": customer_id,
            "merchant": merchant,
            "location": location,
            "amount": amount,
            "timestamp": datetime.now().replace(microsecond=0).isoformat()
        }

        # Insert into PendingCCTransactions
pending_txns_container.create_item(doc)